In [1]:
import random

In [2]:
class TreeNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None

In [3]:
def insert(root, key):
    if not root:
        return TreeNode(key)
    if key < root.key:
        root.left = insert(root.left, key)
    else:
        root.right = insert(root.right, key)
    return root

In [4]:
def delete(root, key):
    if not root:
        return root
    if key < root.key:
        root.left = delete(root.left, key)
    elif key > root.key:
        root.right = delete(root.right, key)
    else:
        if not root.left:
            return root.right
        elif not root.right:
            return root.left
        root.key = minValueNode(root.right)
        root.right = delete(root.right, root.key)
    return root

In [5]:
def minValueNode(node):
    current = node
    while current.left:
        current = current.left
    return current.key

In [6]:
def search(root, key):
    if not root or root.key == key:
        return root
    if key < root.key:
        return search(root.left, key)
    return search(root.right, key)

In [7]:
def getHeight(root):
    if not root:
        return 0
    return 1 + max(getHeight(root.left), getHeight(root.right))

In [8]:
def shuffle_operations(operations):
    random.shuffle(operations)
    return operations

In [9]:
def perform_operations(root, operations):
    metrics = OperationMetrics()
    for operation, key in operations:
        if operation == "search":
            search(root, key)
        elif operation == "insert":
            root = insert(root, key)
        elif operation == "delete":
            root = delete(root, key)
    return metrics

In [10]:
class OperationMetrics:
    def __init__(self):
        self.node_visits = 0
        self.single_rotations = 0
        self.double_rotations = 0
        self.zig_zig_rotations = 0
        self.zig_zag_rotations = 0

In [11]:
def insert_with_metrics(root, key, metrics):
    if not root:
        return TreeNode(key), metrics
    metrics.node_visits += 1
    if key < root.key:
        root.left, child_metrics = insert_with_metrics(root.left, key, metrics)
        metrics.node_visits += child_metrics.node_visits
        metrics.single_rotations += child_metrics.single_rotations
        metrics.double_rotations += child_metrics.double_rotations
        metrics.zig_zig_rotations += child_metrics.zig_zig_rotations
        metrics.zig_zag_rotations += child_metrics.zig_zag_rotations
    else:
        root.right, child_metrics = insert_with_metrics(root.right, key, metrics)
        metrics.node_visits += child_metrics.node_visits
        metrics.single_rotations += child_metrics.single_rotations
        metrics.double_rotations += child_metrics.double_rotations
        metrics.zig_zig_rotations += child_metrics.zig_zig_rotations
        metrics.zig_zag_rotations += child_metrics.zig_zag_rotations
    return root, metrics

In [12]:
def delete_with_metrics(root, key, metrics):
    if not root:
        return root, metrics
    metrics.node_visits += 1
    if key < root.key:
        root.left, child_metrics = delete_with_metrics(root.left, key, metrics)
        metrics.node_visits += child_metrics.node_visits
        metrics.single_rotations += child_metrics.single_rotations
        metrics.double_rotations += child_metrics.double_rotations
        metrics.zig_zig_rotations += child_metrics.zig_zig_rotations
        metrics.zig_zag_rotations += child_metrics.zig_zag_rotations
    elif key > root.key:
        root.right, child_metrics = delete_with_metrics(root.right, key, metrics)
        metrics.node_visits += child_metrics.node_visits
        metrics.single_rotations += child_metrics.single_rotations
        metrics.double_rotations += child_metrics.double_rotations
        metrics.zig_zig_rotations += child_metrics.zig_zig_rotations
        metrics.zig_zag_rotations += child_metrics.zig_zag_rotations
    else:
        metrics.single_rotations += 1  # Rotation during deletion
        if not root.left:
            return root.right, metrics
        elif not root.right:
            return root.left, metrics
        root.key = minValueNode(root.right)
        root.right, child_metrics = delete_with_metrics(root.right, root.key, metrics)
        metrics.node_visits += child_metrics.node_visits
        metrics.single_rotations += child_metrics.single_rotations
        metrics.double_rotations += child_metrics.double_rotations
        metrics.zig_zig_rotations += child_metrics.zig_zig_rotations
        metrics.zig_zag_rotations += child_metrics.zig_zag_rotations
    return root, metrics

In [13]:
def single_rotation_count(node):
    if not node:
        return 0
    return 1 + single_rotation_count(node.left) + single_rotation_count(node.right)

In [14]:
def double_rotation_count(node):
    if not node:
        return 0
    if node.left and node.right:
        return 1 + double_rotation_count(node.left) + double_rotation_count(node.right)
    return double_rotation_count(node.left) + double_rotation_count(node.right)

In [15]:
def zig_zig_rotation_count(node):
    if not node or not node.left or not node.left.left:
        return 0
    return 1 + zig_zig_rotation_count(node.left) + zig_zig_rotation_count(node.left.left)

In [16]:
def zig_zag_rotation_count(node):
    if not node or not node.left or not node.left.right:
        return 0
    return 1 + zig_zag_rotation_count(node.left) + zig_zag_rotation_count(node.left.right)

In [17]:
def conduct_experiment():
    random_numbers = random.sample(range(1, 1000001), 1000000)

    root = None
    for number in random_numbers:
        root = insert(root, number)

    operations = [("search", random.choice(random_numbers)) for _ in range(50000)] + \
                [("insert", random.randint(1000001, 2000000)) for _ in range(50000)] + \
                [("delete", random.choice(random_numbers)) for _ in range(50000)]

    shuffled_operations = shuffle_operations(operations)

    metrics = perform_operations(root, shuffled_operations)

    single_rotations = single_rotation_count(root)
    double_rotations = double_rotation_count(root)
    zig_zig_rotations = zig_zig_rotation_count(root)
    zig_zag_rotations = zig_zag_rotation_count(root)

    normalized_single_rotations = metrics.single_rotations // len(shuffled_operations)
    normalized_double_rotations = metrics.double_rotations // len(shuffled_operations)
    normalized_zig_zig_rotations = metrics.zig_zig_rotations // len(shuffled_operations)
    normalized_zig_zag_rotations = metrics.zig_zag_rotations // len(shuffled_operations)

    return {
        'height': getHeight(root),
        'single_rotations': single_rotations,
        'double_rotations': double_rotations,
        'zig_zig_rotations': zig_zig_rotations,
        'zig_zag_rotations': zig_zag_rotations,
        'normalized_single_rotations': normalized_single_rotations,
        'normalized_double_rotations': normalized_double_rotations,
        'normalized_zig_zig_rotations': normalized_zig_zig_rotations,
        'normalized_zig_zag_rotations': normalized_zig_zag_rotations
    }

In [18]:
num_experiments = 10
average_results = {
    'height': 0,
    'single_rotations': 0,
    'double_rotations': 0,
    'zig_zig_rotations': 0,
    'zig_zag_rotations': 0,
    'normalized_single_rotations': 0,
    'normalized_double_rotations': 0,
    'normalized_zig_zig_rotations': 0,
    'normalized_zig_zag_rotations': 0
}

In [19]:
for _ in range(num_experiments):
    results = conduct_experiment()
    
    for key in average_results.keys():
        average_results[key] += results[key]

In [20]:
for key in average_results.keys():
    average_results[key] /= num_experiments

In [21]:
print("Середні результати:")
print(f"Висота дерева: {average_results['height']}")
print(f"Середня кількість одиничних обертань: {average_results['single_rotations']}")
print(f"Середня кількість подвійних обертань: {average_results['double_rotations']}")
print(f"Середня кількість ZigZig обертань: {average_results['zig_zig_rotations']}")
print(f"Середня кількість ZigZag обертань: {average_results['zig_zag_rotations']}")
print(f"Середня нормована кількість одиничних обертань: {average_results['normalized_single_rotations']}")
print(f"Середня нормована кількість подвійних обертань: {average_results['normalized_double_rotations']}")
print(f"Середня нормована кількість ZigZig обертань: {average_results['normalized_zig_zig_rotations']}")
print(f"Середня нормована кількість ZigZag обертань: {average_results['normalized_zig_zag_rotations']}")

Середні результати:
Висота дерева: 53.8
Середня кількість одиничних обертань: 1001226.2
Середня кількість подвійних обертань: 334373.0
Середня кількість ZigZig обертань: 1795.5
Середня кількість ZigZag обертань: 1681.0
Середня нормована кількість одиничних обертань: 0.0
Середня нормована кількість подвійних обертань: 0.0
Середня нормована кількість ZigZig обертань: 0.0
Середня нормована кількість ZigZag обертань: 0.0
